In [1]:
!pip install elasticsearch
!pip install faiss_cpu
!pip install nlp    
!pip install transformers
!pip install apache_beam



In [2]:
import nlp
eli5 = nlp.load_dataset('eli5')
wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0')['train']

In [3]:
print(eli5)


{'train_eli5': Dataset(features: {'q_id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'selftext': Value(dtype='string', id=None), 'document': Value(dtype='string', id=None), 'subreddit': Value(dtype='string', id=None), 'answers': Sequence(feature={'a_id': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'score': Value(dtype='int32', id=None)}, length=-1, id=None), 'title_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None), 'selftext_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None), 'answers_urls': Sequence(feature={'url': Value(dtype='string', id=None)}, length=-1, id=None)}, num_rows: 272634), 'validation_eli5': Dataset(features: {'q_id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'selftext': Value(dtype='string', id=None), 'document': Value(dtype='string', id=None), 'subreddit': Value(dtype='string', id=None), 'answers': Sequence(featu

In [4]:
pip install git+https://github.com/huggingface/transformers.git


^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/user/anaconda3/lib/python3.11/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/Users/user/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/user/anaconda3/lib/python3.11/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/user/anaconda3/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<fro

In [5]:
import sys
sys.path.append('/Users/user')
from lfqa_utils import *


In [6]:
eli5['test_eli5'][12345]

{'q_id': '8houtx',
 'title': 'Why does water heated to room temperature feel colder than the air around it?',
 'selftext': '',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dylcnfk', 'dylcj49'],
  'text': ["Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder.",
   "Air isn't as good at transferring heat compared to something like water or steel (sit on a room temperature steel bench vs. a room temperature wooden bench, and the steel one will feel more cold).\n\nWhen you feel cold, what you're feeling is heat being transferred out of you.  If there is no breeze, you feel a certain way.  If there's a breeze, you will get colder faster (because the moving air is pulling the heat away from you), and if you get into water, its quite good at pulling heat from you.   Get out of the water and ha

In [7]:
wiki40b_snippets[8991855]


{'_id': '{"nlp_id": 1665419, "wiki_id": "Q179635", "sp": 12, "sc": 653, "ep": 12, "ec": 1223}',
 'article_title': 'Heat transfer',
 'end_character': 1223,
 'end_paragraph': 12,
 'nlp_id': 1665419,
 'passage_text': 'from one place to another place without the movement of particles is called conduction, such as when placing a hand on a cold glass of water - heat is conducted from the warm skin to the cold glass, but if the hand is held a few inches from the glass, little conduction would occur since air is a poor conductor of heat. Steady state conduction is an idealized model of conduction that happens when the temperature difference driving the conduction is constant, so that after a time, the spatial distribution of temperatures in the conducting object does not change any',
 'section_title': 'Conduction',
 'start_character': 653,
 'start_paragraph': 12,
 'wiki_id': 'Q179635'}

In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
)


In [9]:
from elasticsearch import Elasticsearch

# Kết nối đến Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

# Đặt tên chỉ mục
index_name = 'wiki40b_snippets_100w'

# Cấu hình chỉ mục
index_config = {
    "settings": {
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "article_title": {"type": "text"},
            "section_title": {"type": "text"},
            "passage_text": {"type": "text"},
        }
    }
}

# Tạo chỉ mục nếu chưa tồn tại
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_config, headers={"Content-Type": "application/json"})

# Đặt truy vấn và số lượng kết quả
q = 'your query here'  # Đặt truy vấn của bạn vào đây
n_results = 10  # Số lượng kết quả bạn muốn

# Loại bỏ các từ bị cấm
banned = ['how', 'why', 'what', 'where', 'which', 'do', 'does', 'is', '?', 'eli5', 'eli5:']
q = ' '.join([w for w in q.split() if w not in banned])

# Gửi truy vấn tìm kiếm
response = es_client.search(
    index=index_name,
    body={
        "query": {
            "multi_match": {
                "query": q,
                "fields": ["article_title", "section_title", "passage_text^2"],
                "type": "cross_fields",
            }
        },
        "size": n_results,
    }
)

# Xử lý kết quả tìm kiếm
print(response)


{'took': 83, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/ipykernel_24988/1593372765.py:36: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(


In [10]:
if not es_client.indices.exists(index='wiki40b_snippets_100w'):
    make_es_index_snippets(es_client, wiki40b_snippets, index_name='wiki40b_snippets_100w')



In [11]:
def query_es_index(question, es_client, index_name, size=10):
    query_body = {
        "size": size,  # Thêm tham số size vào đây
        "query": {
            "match": {
                "text": question
            }
        }
    }
    
    # Thực hiện truy vấn Elasticsearch
    res = es_client.search(index=index_name, body=query_body)
    
    # Xử lý kết quả trả về
    res_list = [
        {
            'article_title': hit['_source']['article_title'],
            'section_title': hit['_source']['section_title'],
            'passage_text': hit['_source']['passage_text']
        }
        for hit in res['hits']['hits']
    ]
    
    return res, res_list


In [12]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_es_index(question, es_client, index_name='wiki40b_snippets_100w', size=10)


df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/ipykernel_24988/1084859799.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es_client.search(index=index_name, body=query_body)


,Article,Sections,Text
0,---,---,--- Why does water heated to room temperature feel colder than the air around it?


In [1]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
print(f"PyTorch version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
# Set the device      
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(f"Using device: {device}")

PyTorch version: 2.1.2
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [14]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())  # Kiểm tra xem MPS có được hỗ trợ không


2.1.2
True


In [19]:
device = torch.device("mps")


In [2]:
import torch
import os

class ArgumentsQAR():
    def __init__(self):
        self.batch_size = 512
        self.max_length = 128
        self.checkpoint_batch_size = 32
        self.print_freq = 100
        self.pretrained_model_name = "google/bert_uncased_L-8_H-768_A-12"
        self.model_save_name = "retriever_models/eli5_retriever_model_l-8_h-768_b-512-512"
        self.learning_rate = 2e-4
        self.num_epochs = 10

qar_args = ArgumentsQAR()

# Tạo thư mục để lưu mô hình nếu chưa tồn tại
if not os.path.exists('retriever_models'):
    os.makedirs('retriever_models')

# Hàm lưu checkpoint
def save_checkpoint(model, optimizer, scheduler, epoch, model_save_name):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }
    
    torch.save(checkpoint, f"{model_save_name}_checkpoint_{epoch}.pth")
    print(f"Checkpoint saved at epoch {epoch}")

# Hàm tải lại checkpoint
def load_checkpoint(model, optimizer, scheduler, model_save_name, epoch):
    checkpoint = torch.load(f"{model_save_name}_checkpoint_{epoch}.pth", map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"Checkpoint loaded from epoch {epoch}")
    return checkpoint['epoch']

# Chuẩn bị dataset (bạn cần định nghĩa hoặc import ELI5DatasetQARetriver)
# Đảm bảo rằng ELI5DatasetQARetriver đã được định nghĩa trước đó
qar_train_dset = ELI5DatasetQARetriver(eli5['train_eli5'], training=True)
qar_valid_dset = ELI5DatasetQARetriver(eli5['validation_eli5'], training=False)

# Tải mô hình BERT đã huấn luyện trước
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

qar_tokenizer, qar_model = make_qa_retriever_model(
    model_name=qar_args.pretrained_model_name,
    from_file=None,
    device="mps"  # Sử dụng MPS
)

# Khởi tạo optimizer và scheduler
qar_optimizer = torch.optim.AdamW(qar_model.parameters(), lr=qar_args.learning_rate)
qar_scheduler = torch.optim.lr_scheduler.StepLR(qar_optimizer, step_size=1)

# Khôi phục từ checkpoint (nếu có checkpoint đã lưu)
last_epoch = 0  # Bắt đầu từ epoch 0
try:
    last_epoch = load_checkpoint(qar_model, qar_optimizer, qar_scheduler, qar_args.model_save_name, 532)  # Sử dụng epoch 532
except FileNotFoundError:
    print("Checkpoint không tồn tại, bắt đầu huấn luyện từ epoch 0.")

# Huấn luyện mô hình từ epoch đã lưu
for e in range(last_epoch, qar_args.num_epochs):
    train_qa_retriever(qar_model, qar_tokenizer, qar_train_dset, qar_valid_dset, qar_args)
    save_checkpoint(qar_model, qar_optimizer, qar_scheduler, e, qar_args.model_save_name)
    

NameError: name 'ELI5DatasetQARetriver' is not defined

In [17]:
 qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('mps')
_ = qar_model.eval()

config.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/325M [00:00<?, ?B/s]

AttributeError: 'RetriBertConfig' object has no attribute 'position_embedding_type'